In [2]:
import requests
import pandas as pd
import json
from datetime import datetime, timedelta
import time
import tkinter as tk
from tkinter import messagebox
import statistics

In [3]:
#Root function creates a message box that creates an alert when market conditions are met
root = tk.Tk()
root.withdraw()

''

In [4]:
#API code is provided free by Alphavantage, it is limited to 5 requests a minute.
#Having a premium API key can allow us to watchlist multiple stocks at the same time.
api_key = '1G3JSDK3B4WMKWE'

#Stock that you would like to watch is created by input or can be hard coded into the script. 
#You can use both Upper and lower case letters.
symbol = input('Please enter the exact ticker name that you would like to track ').upper()
interval = 'daily_adjusted'

#Price tolerance is a threshold percentage of how much higher can price be than the 50 day moving average before alert is triggered.
#Volume tolerance is a threshold percentage of how much higher can volume spike above average daily volume before alert is triggered.
price_tolerance = 1.6
volume_tolerance = 1.4

#Alphavantage API provides multiple functions, to see more visit: "https://www.alphavantage.co/documentation/"
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_{interval}&symbol={symbol}&apikey={api_key}'

response = requests.get(url)
data = json.loads(response.text)

Please enter the exact ticker name that you would like to track AAPL


In [6]:
# Extract the monthly time series data
data = response.json()[f"Time Series (Daily)"]
df = pd.DataFrame.from_dict(data, orient="index")
df.index = pd.to_datetime(df.index)

df.head(7)

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
2023-03-30,161.53,162.47,161.27,162.36,162.36,49459330,0.0000,1.0
2023-03-29,159.37,161.05,159.35,160.77,160.77,51305691,0.0000,1.0
2023-03-28,157.97,158.49,155.98,157.65,157.65,45992152,0.0000,1.0
2023-03-27,159.94,160.77,157.87,158.28,158.28,52390266,0.0000,1.0
2023-03-24,158.86,160.34,157.85,160.25,160.25,59256343,0.0000,1.0
2023-03-23,158.83,161.5501,157.68,158.93,158.93,67622060,0.0000,1.0
2023-03-22,159.3,162.14,157.81,157.83,157.83,75701811,0.0000,1.0


In [9]:
# Calculating 50 day moving average of the stock ending with the current date.
moving_avg = df['4. close'][::-1].rolling(window=50).mean()
m_avg=float(moving_avg.tail(1))

In [10]:
# Calculating average 50 day volume to compare against the current day's volume.
volume=list(df['6. volume'][:50])
volume_avg = 0
for i in volume:
    volume_avg += int(i)
volume_avg = int(volume_avg/50)
volume_spike_tolerance = volume_avg*volume_tolerance

In [11]:
# API request to get the current quote on volume and price of a stock
url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={api_key}'
response=requests.get(url)
quote=json.loads(response.text)
print(quote)

{'Global Quote': {'01. symbol': 'AAPL', '02. open': '161.5300', '03. high': '162.4700', '04. low': '161.2700', '05. price': '162.3600', '06. volume': '49459330', '07. latest trading day': '2023-03-30', '08. previous close': '160.7700', '09. change': '1.5900', '10. change percent': '0.9890%'}}


In [ ]:
while True:
    # Make a request to the API
    response = requests.get(url)

    # Parse the JSON response
    quote = json.loads(response.text)

    # Get the latest stock price
    current_price = quote ['Global Quote']['05. price']
    volume_quote = quote ['Global Quote']['06. volume']

    # Print the latest stock price
    print(f'Current {symbol} stock price: ${current_price}')

    #Warning if the current price is higher than the moving average by over 40%, 
    #as well as volume spike is above 40% of the 50 day average.
    if float(current_price)*price_tolerance>=m_avg and volume_quote>=volume_spike_tolerance:
        messagebox.showinfo("Alert", "Squeeze is Happening")
        break

    # Wait for 1 minute before making the next request
    time.sleep(60)

Current AAPL stock price: $162.3600
